In [ ]:
!pip -q install ydata_profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.8 MB/s eta 0:00:00


In [80]:
import pandas as pd
import numpy as np
import re
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
# from ydata_profiling import ProfileReport


class Page:
    """
    Represents a webpage and provides methods to extract specific text elements.
    """

    def __init__(self, r_content):
        """
        Initializes the Page object with the HTML content.

        Args:
            r_content: The HTML content as bytes.
        """
        self.soup = BeautifulSoup(r_content, 'html.parser')
        for article in self.soup.find_all('article'):
            article.decompose()
        for ad in self.soup.find_all('div', class_='css-ukcqzp'):
            ad.decompose()
        self._title = None  # Initialize private variable for topic
        self._question = None  # Initialize private variable for question
        self._sa = None  # Initialize private variable for short answer
        self._la = None  # Initialize private variable for long answer
        self._lb = None  # Initialize private variable for legal basis

    def _get_text_content(self, selector, tag_name='p'):
        """
        Extracts text content from a container identified by the selector (private method).

        Args:
            selector: A dictionary representing the selector criteria (e.g., class_, id).
            tag_name: The tag name to find within the container (default: 'p').

        Returns:
            The extracted text content (stripped), or None if not found.
        """
        container = self.soup.find(**selector)
        return container.contents[0].text.strip() if container else None

    def _get_filtered_text_content(self, selector=None, parent=None, tag_name='p', filter_words=None):
        """
        Extracts text content from a container, filtering out specific words (private method).

        Args:
            selector: A dictionary representing the selector criteria (e.g., class_, id). Defaults to None.
            parent: The parent element to search within (optional). Defaults to None.
            tag_name: The tag name to find within the container (default: 'p').
            filter_words: A list of words to exclude from the extracted content.

        Returns:
            A list of extracted text content (stripped) for each paragraph,
            excluding the filter words. Empty list if not found.
        """
        container = None
        if selector:
            container = self.soup.find(**selector)
        elif parent:
            container = parent  # Use the provided parent element

        if container:
            paragraphs = container.find_all(tag_name)
            return [tag.text.strip() for tag in paragraphs if all(word not in tag.text.strip() for word in filter_words)]
        return []

    def get_title(self):
        """
        Extracts and returns the title of the page (cached).
        """
        if self._title is None:
            title_selector = {'class_': 'css-12b9o33 e97iaci2'}
            self._title = self._get_text_content(title_selector)
            print("Found title!") if self._title else print("No title found.")
        return self._title

    def get_q(self):
        """
        Extracts and returns the question text from the page (cached).
        """
        if self._question is None:
            question_selector = {'class_': 'css-c816ma e1vjmfpm0'}
            self._question = self._get_text_content(question_selector)
            print("Found question!") if self._question else print("No question found.")
        return self._question

    def get_sa(self):
        """
        Extracts and returns the short answer text from the page (cached).
        """
        if self._sa is None:
            sa = self.soup.find_all('p', class_='css-dvbmnz')
            print("Found short answer!") if sa else print("No short answer found.")
            if sa:
                sa = [tag.text.strip() for tag in sa]
                self._sa = ' '.join(sa[:-1])
            else:
                self._sa = None
        return self._sa

    def get_la(self):
        """
        Extracts and returns the filtered long answer text from the page (cached).
        """
        if self._la is None:
            selectors = [
                ('p', {'string': 'Terima kasih atas pertanyaan Anda.'}),
                ('p', {'string': 'Terimakasih atas pertanyaan Anda.'}),
                ('div', {'class_': 'css-19ozjwg ejhsnq52'}),
                ('div', {'class_': 'css-9y96r6 elbhtsw1'}),
                # ('div', {'class_': 'css-103zlhi elbhtsw0'})
            ]

            # Try each selector until we find a match
            for tag, attrs in selectors:
                if found_tag := self.soup.find(tag, **attrs):
                    unique_tag = found_tag
            if unique_tag:
                la_container = unique_tag.parent  # Get the parent container
                # la_content = la_container.find_all('p')  # Find all paragraph elements within the container
                # self._la = [p.text.strip() for p in la_content if p.text.strip() and all(word not in p.text.strip() for word in ['Terima kasih atas pertanyaan Anda', 'Artikel di bawah ini adalah pemutakhiran', 'Seluruh informasi hukum yang ada di Klinik hukumonline.com', 'Baca juga'])]  # Filter content and cache
                self._la = la_container.get_text(separator=' ', strip=True)
                patterns_to_remove = [
                    'terima kasih atas pertanyaan anda\.',
                    'artikel di bawah ini adalah pemutakhiran',
                    'baca juga',
                    'seluruh informasi hukum yang ada di klinik hukumonline\.com .*?\.',
                    'untuk mendapatkan nasihat hukum spesifik terhadap kasus anda, konsultasikan langsung dengan konsultan mitra justika',
                    'klinik terkait'
                ]
                combined_pattern = '|'.join(patterns_to_remove)
                self._la = re.sub(combined_pattern, '', self._la, flags=re.IGNORECASE)
            else:
                self._la = ''  # Set empty list if not found
        return self._la.strip()


    def get_lb(self):
        """
        Extracts and returns a list of legal basis text snippets from the page (cached).
        """
        dasar = None
        if self._lb is None:
            lb_content = [] # container
            detector1 = self.soup.find('u', string=lambda text: text and "dasar hukum" in text.lower()) # find first u tag with string 'dasar hukum'
            detector2 = self.soup.find('h2', string=lambda text: text and "referensi" in text.lower()) # find first u tag with string 'Referensi'55
            dasar = detector1 or detector2 # find first u tag with string 'dasar hukum' or 'Referensi'
            number_pattern = r"\[\d+\]"  # Matches strings like '[1]', '[4]', etc.
            if dasar:
                anchor_tags = dasar.find_all_next('a', string=lambda text: text and re.search(number_pattern, text))
                aunty = dasar.parent.find_next_sibling()
                if anchor_tags:
                    for anchor in anchor_tags:
                        lb = anchor.parent
                        lb_content.extend([lb.text.strip()])
                elif aunty and (aunty.name == 'ol' or aunty.name == 'ul'):
                    for child in aunty:
                        lb_content.extend([child.text.strip()])
                else :
                    lb_content = None
            else:
                lb_content = None

            self._lb = lb_content

        return self._lb


def remove_duplicates(list2d):
  """
  Removes duplicate elements from a 2D list.

  Args:
      list2d: The 2D list to remove duplicates from.

  Returns:
      A new 2D list with duplicates removed.
  """

  # Convert the 2D list to a set to remove duplicates.
  unique_set = set(tuple(row) for row in list2d)

  # Convert the set back to a 2D list.
  unique_list = [list(row) for row in unique_set]

  return unique_list

In [81]:
# url = 'https://www.hukumonline.com/klinik/a/larangan-merokok-di-pesawat-lt655dc4bb6d9c4/'

# response = requests.get(url)
# page = Page(response.content)
# page.get_la()
all = pd.read_csv('/content/drive/MyDrive/SKRIPSI/clean_v0.35_1118.csv', index_col=0)
urls = all['link'].to_list()
for url in tqdm(urls):
    response = requests.get(url)
    page = Page(response.content)
    all.loc[all['link'] == url, 'long_answer'] = page.get_la()
#

100%|██████████| 1118/1118 [07:18<00:00,  2.55it/s]


In [79]:
url = 'https://www.hukumonline.com/klinik/a/apa-perbedaan-delik-formil-dan-delik-materil-lt569f12361488b/'

response = requests.get(url)
page = Page(response.content)
page.get_la()

<div class="css-9y96r6 elbhtsw1"><svg fill="none" height="32" viewbox="0 0 32 32" width="32" xmlns="http://www.w3.org/2000/svg"><path d="M28 9.33317V22.6665C28 26.6665 26 29.3332 21.3333 29.3332H10.6667C6 29.3332 4 26.6665 4 22.6665V9.33317C4 5.33317 6 2.6665 10.6667 2.6665H21.3333C26 2.6665 28 5.33317 28 9.33317Z" fill="#89BFF4" opacity="0.4"></path><path d="M20.6666 2.6665V13.1465C20.6666 13.7332 19.9732 14.0265 19.5465 13.6399L16.4533 10.7865C16.2 10.5465 15.7999 10.5465 15.5465 10.7865L12.4533 13.6399C12.0266 14.0399 11.3333 13.7332 11.3333 13.1465V2.6665H20.6666Z" fill="#003366"></path><path d="M23.3334 19.6665H17.6667C17.1201 19.6665 16.6667 19.2132 16.6667 18.6665C16.6667 18.1198 17.1201 17.6665 17.6667 17.6665H23.3334C23.8801 17.6665 24.3334 18.1198 24.3334 18.6665C24.3334 19.2132 23.8801 19.6665 23.3334 19.6665Z" fill="#003366"></path><path d="M23.3333 25H12C11.4533 25 11 24.5467 11 24C11 23.4533 11.4533 23 12 23H23.3333C23.88 23 24.3333 23.4533 24.3333 24C24.3333 24.5467 23.8

'ULASAN LENGKAP Terimakasih atas pertanyaan Anda.  dari artikel dengan judul Cara Membedakan Delik Formil dan Delik Materil yang dibuat oleh Yuda Pencawan, S.H. dari Lembaga Bantuan Hukum Mawar Saron yang pertama kali dipublikasikan pada 9 Mei 2016. Artikel ini dibuat berdasarkan KUHP lama dan UU 1/2023 tentang KUHP yang diundangkan pada tanggal 2 Januari 2023.   . Apa itu Delik? Sebelumnya, penting untuk kita ketahui bahwa delik merupakan istilah yang berasal dari bahasa latin yaitu “ delictum ”. Delik dapat diartikan sebagai tindak pidana, peristiwa pidana, atau perbuatan pidana. [1] Adapun tindak pidana merupakan terjemahan dari bahasa Belanda, yaitu strafbaar feit sebagai suatu perbuatan manusia yang dapat dihukum. [2] Berdasarkan pendapat Pompe yang dikutip Andi Sofyan dan Nur Azisa dalam bukunya buku Hukum Pidana (hal. 98), tindak pidana adalah suatu pelanggaran norma (gangguan terhadap tertib hukum) yang dengan sengaja ataupun tidak dengan sengaja telah dilakukan oleh pelaku, di

In [84]:
all.to_csv('/content/drive/MyDrive/SKRIPSI/clean_v0.36_1118.csv')

# Get Questions & Answers

##Scraping and parsing [--pidana--]

In [ ]:
#loop over pages
r = requests.get('https://www.hukumonline.com/klinik/pidana/page/')
page_infos = []

for page_num in tqdm(range(1, 161)):
    print(f'\n===Page {page_num} : {page_num*10-10}-{page_num*10}===')
    r = requests.get(f'https://www.hukumonline.com/klinik/pidana/page/{page_num}/')
    soup = BeautifulSoup(r.content, 'html.parser')
    posts = soup.find_all('div', class_='klinik-list')
    if posts:
        print('found "div/klinik-list"')
        for post in posts:
            page_info = []
            anchor = post.find('a')
            post_req = requests.get(f'https://www.hukumonline.com{anchor["href"]}')
            if post_req.status_code == 200:
                print(f'Page https://www.hukumonline.com{anchor["href"]} : request success')
                res = Page(post_req.content)

                #Get questions
                post_t = res.get_title()
                page_info.append(post_t)

                #Get questions
                post_q = res.get_q()
                page_info.append(post_q)

                #Get short answer
                post_sa = res.get_sa()
                page_info.append(post_sa)

                #Get long answer
                post_la = res.get_la()
                #concat list la to single long string
                post_la_str = ' '.join(post_la)
                page_info.append(post_la_str)

                #Get legal basis
                post_lb = res.get_lb()
                page_info.append(post_lb)

                #Get link
                page_info.append(f'https://www.hukumonline.com{anchor["href"]}')

                page_infos.append(page_info)
            else:
                print(f'Page https://www.hukumonline.com{anchor["href"]} request failed; status code: ', post_req.status_code)

# to pandas df with cols : ['question', 'short_answer', 'long_answer', 'legal_basis]
df = pd.DataFrame(page_infos, columns=['topic','question', 'short_answer', 'long_answer', 'legal_basis', 'link'])
df

  0%|          | 0/160 [00:00<?, ?it/s]


===Page 1 : 0-10===
found "div/klinik-list"
Page https://www.hukumonline.com/klinik/a/jerat-pasal-penyalahgunaan-dana-donasi-lt670e620d628a9/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/pengertian-iovermacht-i-dan-iultimum-remedium-i-dalam-hukum-pidana-lt56dd07c1a7dbf/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/kdrt-hingga-meninggal--penganiayaan-atau-pembunuhan-lt602efcc14f6c0/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/pacar-tidak-mau-bertanggung-jawab--kapan-maksimal-bisa-dilaporkan-lt53b2e5ad1e75a/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/istri-siri-selingkuh--bisakah-dipidana-lt66b659c5509fe/ : request success
Found title!
Found question!
Found 

  1%|          | 1/160 [00:12<32:55, 12.42s/it]


===Page 2 : 10-20===
found "div/klinik-list"
Page https://www.hukumonline.com/klinik/a/jerat-pidana-pelaku-mutilasi-dan-pemerkosa-mayat-lt567ac0977c701/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/siapa-pihak-yang-menilai-kerugian-keuangan-negara-dalam-tipikor-lt57d7ba0cd90bb/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/bisakah-pemberian-grasi-dicabut-kembali-lt50a9c016d00f3/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/pasal-56-kuhp-tentang-pembantuan-tindak-pidana-lt66f21b592a715/ : request success
Found title!
Found question!
Found short answer!
Found long answer!
Page https://www.hukumonline.com/klinik/a/apakah-kekerasan-oleh-kakak-ipar-termasuk-kdrt-lt66ec7c312a715/ : request success
Found title!
Found question!
Found short answer!
F

  1%|▏         | 2/160 [00:20<26:00,  9.88s/it]

...

 99%|█████████▉| 159/160 [27:55<00:12, 12.41s/it]

Page https://www.hukumonline.com/klinik/a/bisakah-masa-hukuman-istri-digantikan-oleh-suaminya--lt4fcb312e9dd57/ : request success
Found title!
Found question!
No short answer found.
No long answer found.
Found alternative long answer selector

===Page 160 : 1590-1600===
found "div/klinik-list"
Page https://www.hukumonline.com/klinik/a/perlindungan-saksi-dan-korban-lt4fc83ea563cc9/ : request success
Found title!
Found question!
No short answer found.
No long answer found.
Found alternative long answer selector
Page https://www.hukumonline.com/klinik/a/hukuman-lt4fbddb474ff92/ : request success
Found title!
Found question!
No short answer found.
No long answer found.
Found alternative long answer selector
Page https://www.hukumonline.com/klinik/a/perlindungan-saksi-dan-korban-lt4fbc7b673bc18/ : request success
Found title!
Found question!
No short answer found.
No long answer found.
Found alternative long answer selector
Page https://www.hukumonline.com/klinik/a/apakah-ini-penggelapan--l

100%|██████████| 160/160 [28:08<00:00, 10.55s/it]

Page https://www.hukumonline.com/klinik/a/cara-pembuktian-ancaman-yang-dilakukan-melalui-pesan-blackberry-messenger-bbm--lt4f7a6b0b86a1f/ : request success
Found title!
Found question!
No short answer found.
No long answer found.
Found alternative long answer selector


,topic,question,short_answer,long_answer,legal_basis,link
0,Jerat Pasal Penyalahgunaan Dana Donasi,Apa pasal yang bisa menjerat pengurus yayasan ...,Penyalahgunaan dana donasi oleh pengurus yayas...,Untuk menjawab pertanyaan Anda mengenai apakah...,[[1] Penjelasan Pasal 32 ayat (1) Undang-Undan...,https://www.hukumonline.com/klinik/a/jerat-pas...
1,Pengertian,Bagaimana ketentuan overmacht dalam hukum pida...,Daya paksa atau overmacht dalam hukum pidana m...,"11 Jan, 2023 15 Jul, 2022 10 Jul, 2020 22 Okt,...",[[1] R. Sugandhi. Kitab Undang-Undang Hukum Pi...,https://www.hukumonline.com/klinik/a/pengertia...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...","Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,"01 Mar, 2024 18 Nov, 2020 08 Okt, 2020 02 Agt,...",[[1] Pasal 1 angka 1 Undang-Undang Nomor 23 Ta...,https://www.hukumonline.com/klinik/a/kdrt-hing...
3,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,"22 Jul, 2024 12 Jun, 2024 19 Apr, 2024 09 Okt,...",[[1] Pasal 624 Undang-Undang Nomor 1 Tahun 202...,https://www.hukumonline.com/klinik/a/pacar-tid...
4,"Istri Siri Selingkuh, Bisakah Dipidana?",Saya seorang pria yang sudah beristri. Sayangn...,Perselingkuhan yang melibatkan pihak yang suda...,Artikel ini dibuat berdasarkan KUHP lama dan U...,[[1] Pasal 1 Undang-Undang Nomor 1 Tahun 1974 ...,https://www.hukumonline.com/klinik/a/istri-sir...
...,...,...,...,...,...,...
1595,Hambatan-hambatan Pelaksanaan Perlindungan Sak...,Apa saja hambatan atau masalah dalam menerapka...,None,Hambatan pelaksanaan perlindungan saksi dan ko...,None,https://www.hukumonline.com/klinik/a/hambatan-...
1596,Hukuman Bagi Wanita yang Selingkuh dan Ingin A...,"Saya wanita bersuami, tapi saya berselingkuh d...",None,Selingkuh dan aborsi merupakan dua jenis tinda...,None,https://www.hukumonline.com/klinik/a/perseling...
1597,Proses Hukum Jika Anggota TNI Memukul Warga,"Saya ingin menanyakan, bagaimanakah hukumnya a...",None,Kami menyampaikan terima kasih atas pertanyaan...,None,https://www.hukumonline.com/klinik/a/tni-yang-...
1598,"Orang Tua yang Izinkan Anaknya Berselingkuh, B...",Seorang ibu yang izinkan anaknya selingkuh apa...,None,Dalam hukum pidana yang digolongkan/dianggap s...,None,https://www.hukumonline.com/klinik/a/perseling...


In [ ]:
from datetime import datetime

csv_name = f'raw_{len(df)}_{datetime.now().strftime("%d-%m-%Y_%H:%M")}'
df.to_csv(f'/content/drive/MyDrive/SKRIPSI/{csv_name}.csv')
print(f'Saved to {csv_name}.csv')

Saved to raw_1600_16-10-2024_03:03.csv


In [ ]:
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/raw_1600_16-10-2024_03:03.csv')
# drop Unnamed cols if exist
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Drop empty lbs
df = df[df['legal_basis'].notna()]
df = df.reset_index(drop=True)

# csv_name = f'clean_{len(df)}_{datetime.now()}'
# df.to_csv(f'/content/drive/MyDrive/SKRIPSI/{csv_name}.csv')

# print(f'Saved to {csv_name}.csv')

df

,topic,question,short_answer,long_answer,legal_basis,link
0,Jerat Pasal Penyalahgunaan Dana Donasi,Apa pasal yang bisa menjerat pengurus yayasan ...,Penyalahgunaan dana donasi oleh pengurus yayas...,Untuk menjawab pertanyaan Anda mengenai apakah...,['[1] Penjelasan Pasal 32 ayat (1) Undang-Unda...,https://www.hukumonline.com/klinik/a/jerat-pas...
1,Pengertian,Bagaimana ketentuan overmacht dalam hukum pida...,Daya paksa atau overmacht dalam hukum pidana m...,"11 Jan, 2023 15 Jul, 2022 10 Jul, 2020 22 Okt,...",['[1] R. Sugandhi.\xa0Kitab Undang-Undang Huku...,https://www.hukumonline.com/klinik/a/pengertia...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...","Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,"01 Mar, 2024 18 Nov, 2020 08 Okt, 2020 02 Agt,...",['[1] Pasal 1 angka 1 Undang-Undang Nomor 23 T...,https://www.hukumonline.com/klinik/a/kdrt-hing...
3,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,"22 Jul, 2024 12 Jun, 2024 19 Apr, 2024 09 Okt,...",['[1] Pasal 624 Undang-Undang Nomor 1 Tahun 20...,https://www.hukumonline.com/klinik/a/pacar-tid...
4,"Istri Siri Selingkuh, Bisakah Dipidana?",Saya seorang pria yang sudah beristri. Sayangn...,Perselingkuhan yang melibatkan pihak yang suda...,Artikel ini dibuat berdasarkan KUHP lama dan U...,['[1] Pasal 1 Undang-Undang Nomor 1 Tahun 1974...,https://www.hukumonline.com/klinik/a/istri-sir...
...,...,...,...,...,...,...
1123,Membayar Lomba Lari dengan Mata Uang Asing,Saya pernah dengar ada kewajiban menggunakan m...,NaN,Jika lomba lari tersebut diselenggarakan di In...,['Undang-Undang Nomor 7 Tahun 2011 tentang Mat...,https://www.hukumonline.com/klinik/a/membayar-...
1124,Larangan Jual Beli Darah,Apakah perdagangan organ tubuh dilarang? Tapi ...,NaN,"14 Agt, 2024 10 Mar, 2014 11 Apr, 2013 31 Okt,...",['Undang-Undang Nomor 36 Tahun 2009 tentang Ke...,https://www.hukumonline.com/klinik/a/larangan-...
1125,Menjerat Pelaku Kampanye Hitam di Media Sosial,Mulai memanasnya situasi menjelang PilPres 201...,NaN,Kampanye dalam Pemilihan Umum Presiden (Pilpre...,['Undang-Undang No. 11 Tahun 2008 tentang Info...,https://www.hukumonline.com/klinik/a/menjerat-...
1126,Eksekusi Hukuman Mati Bagi Terpidana yang Hamil,Jika ada kasus seorang isteri perwira TNI AD m...,NaN,"04 Jul, 2024 30 Apr, 2024 17 Jan, 2023 23 Des,...","['Kitab Undang-Undang Hukum Pidana', 'Undang-U...",https://www.hukumonline.com/klinik/a/eksekusi-...


In [ ]:
print("Empty q = ", len(df[df['question'].isna()]))
print("Empty sa = ", len(df[df['short_answer'].isna()]))
print("Empty la = ", len(df[df['long_answer'].isna()]))
print("Empty lb = ", len(df[df['legal_basis'].isna()]))

Empty q =  114
Empty sa =  492
Empty la =  10
Empty lb =  0


In [ ]:
link_of_empty_q_s = df.loc[df['question'].isna(), 'link']
link_of_empty_q_s

,link
42,https://www.hukumonline.com/klinik/a/kembalika...
90,https://www.hukumonline.com/klinik/a/cara-meng...
937,https://www.hukumonline.com/klinik/a/langkah-h...
938,https://www.hukumonline.com/klinik/a/pidana-ba...
939,https://www.hukumonline.com/klinik/a/dasar-huk...
...,...
1078,https://www.hukumonline.com/klinik/a/keberlaku...
1079,https://www.hukumonline.com/klinik/a/penyelesa...
1080,https://www.hukumonline.com/klinik/a/tata-cara...
1082,https://www.hukumonline.com/klinik/a/tertangka...


In [ ]:
def alt_get_q(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    question_selector = {'class_': 'css-c816ma e1vjmfpm0'}
    question = soup.find(**question_selector)
    q_text = question.find_all('p')
    q_text = [tag.text.strip() for tag in q_text]
    return ' '.join(q_text)

links_to_fill = []
for link in link_of_empty_q_s:
    links_to_fill.append(alt_get_q(link))

links_to_fill

In [ ]:
df_filled_q = df.copy()
df_filled_q.loc[df_filled_q['question'].isna(), 'question'] = links_to_fill
df_filled_q

,topic,question,short_answer,long_answer,legal_basis,link
0,Jerat Pasal Penyalahgunaan Dana Donasi,Apa pasal yang bisa menjerat pengurus yayasan ...,Penyalahgunaan dana donasi oleh pengurus yayas...,Untuk menjawab pertanyaan Anda mengenai apakah...,['[1] Penjelasan Pasal 32 ayat (1) Undang-Unda...,https://www.hukumonline.com/klinik/a/jerat-pas...
1,Pengertian,Bagaimana ketentuan overmacht dalam hukum pida...,Daya paksa atau overmacht dalam hukum pidana m...,"11 Jan, 2023 15 Jul, 2022 10 Jul, 2020 22 Okt,...",['[1] R. Sugandhi.\xa0Kitab Undang-Undang Huku...,https://www.hukumonline.com/klinik/a/pengertia...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...","Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,"01 Mar, 2024 18 Nov, 2020 08 Okt, 2020 02 Agt,...",['[1] Pasal 1 angka 1 Undang-Undang Nomor 23 T...,https://www.hukumonline.com/klinik/a/kdrt-hing...
3,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,"22 Jul, 2024 12 Jun, 2024 19 Apr, 2024 09 Okt,...",['[1] Pasal 624 Undang-Undang Nomor 1 Tahun 20...,https://www.hukumonline.com/klinik/a/pacar-tid...
4,"Istri Siri Selingkuh, Bisakah Dipidana?",Saya seorang pria yang sudah beristri. Sayangn...,Perselingkuhan yang melibatkan pihak yang suda...,Artikel ini dibuat berdasarkan KUHP lama dan U...,['[1] Pasal 1 Undang-Undang Nomor 1 Tahun 1974...,https://www.hukumonline.com/klinik/a/istri-sir...
...,...,...,...,...,...,...
1123,Membayar Lomba Lari dengan Mata Uang Asing,Saya pernah dengar ada kewajiban menggunakan m...,NaN,Jika lomba lari tersebut diselenggarakan di In...,['Undang-Undang Nomor 7 Tahun 2011 tentang Mat...,https://www.hukumonline.com/klinik/a/membayar-...
1124,Larangan Jual Beli Darah,Apakah perdagangan organ tubuh dilarang? Tapi ...,NaN,"14 Agt, 2024 10 Mar, 2014 11 Apr, 2013 31 Okt,...",['Undang-Undang Nomor 36 Tahun 2009 tentang Ke...,https://www.hukumonline.com/klinik/a/larangan-...
1125,Menjerat Pelaku Kampanye Hitam di Media Sosial,Mulai memanasnya situasi menjelang PilPres 201...,NaN,Kampanye dalam Pemilihan Umum Presiden (Pilpre...,['Undang-Undang No. 11 Tahun 2008 tentang Info...,https://www.hukumonline.com/klinik/a/menjerat-...
1126,Eksekusi Hukuman Mati Bagi Terpidana yang Hamil,Jika ada kasus seorang isteri perwira TNI AD m...,NaN,"04 Jul, 2024 30 Apr, 2024 17 Jan, 2023 23 Des,...","['Kitab Undang-Undang Hukum Pidana', 'Undang-U...",https://www.hukumonline.com/klinik/a/eksekusi-...


In [ ]:
df_filled_q.to_csv(f'/content/drive/MyDrive/SKRIPSI/q_filled_{len(df_filled_q)}.csv')
print(f'Saved to q_filled_{len(df_filled_q)}.csv')

In [ ]:
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/q_filled_1128.csv')

In [ ]:
wrong_la_link = df.loc[df['long_answer'].str.len() <= 60, 'link']
wrong_la_link

,link
611,https://www.hukumonline.com/klinik/a/jerat-huk...
614,https://www.hukumonline.com/klinik/a/sanksi-ba...
620,https://www.hukumonline.com/klinik/a/hukumnya-...
621,https://www.hukumonline.com/klinik/a/hukumnya-...
626,https://www.hukumonline.com/klinik/a/hukumnya-...
...,...
929,https://www.hukumonline.com/klinik/a/benarkah-...
930,https://www.hukumonline.com/klinik/a/apa-alasa...
931,https://www.hukumonline.com/klinik/a/yang-meny...
932,https://www.hukumonline.com/klinik/a/bisakah-d...


In [ ]:
def alt_get_la(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    la_selector = {'class_': 'css-19ozjwg ejhsnq52'}
    la = soup.find(**la_selector)
    if la:
        la_text = la.parent.find_all('div')
        la_text = [tag.text.strip() for tag in la_text]
        print('Success ', url)
        return ' '.join(la_text)
    else:
        return None

la_to_fill = []
for link in tqdm(wrong_la_link):
    la_to_fill.append(alt_get_la(link))

la_to_fill

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df.loc[df['long_answer'].str.len() <= 60, 'long_answer'] = la_to_fill
df.to_csv(f'/content/drive/MyDrive/SKRIPSI/la_fixedddddd_{len(df)}.csv')
print(f'Saved to la_fixed_{len(df)}.csv')

Saved to la_fixed_1128.csv


In [ ]:
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/raw_data/la_fixedddddd_1128.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('Unnamed: 0.1', axis=1)
df.to_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.1_{len(df)}.csv')
print(f'Saved to clean_v0.1_{len(df)}.csv')

Saved to clean_v0.1_1128.csv


In [ ]:
print("Empty q = ", len(df[df['question'].isna()]))
print("Empty sa = ", len(df[df['short_answer'].isna()]))
print("Empty la = ", len(df[df['long_answer'].isna()]))
print("Empty lb = ", len(df[df['legal_basis'].isna()]))

Empty q =  0
Empty sa =  492
Empty la =  10
Empty lb =  0


In [ ]:
df = df.dropna(subset=['long_answer'])
df = df.reset_index(drop=True)
df

In [ ]:
df.to_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.2_{len(df)}.csv')
print(f'Saved to clean_v0.2_{len(df)}.csv')

Saved to clean_v0.2_1118.csv


# Get Peraturan / Legal Basis (LB)

In [ ]:
import pandas as pd
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.2_1118.csv', index_col=0)
df

,topic,question,short_answer,long_answer,legal_basis,link
0,Jerat Pasal Penyalahgunaan Dana Donasi,Apa pasal yang bisa menjerat pengurus yayasan ...,Penyalahgunaan dana donasi oleh pengurus yayas...,Untuk menjawab pertanyaan Anda mengenai apakah...,['[1] Penjelasan Pasal 32 ayat (1) Undang-Unda...,https://www.hukumonline.com/klinik/a/jerat-pas...
1,Pengertian,Bagaimana ketentuan overmacht dalam hukum pida...,Daya paksa atau overmacht dalam hukum pidana m...,"11 Jan, 2023 15 Jul, 2022 10 Jul, 2020 22 Okt,...",['[1] R. Sugandhi.\xa0Kitab Undang-Undang Huku...,https://www.hukumonline.com/klinik/a/pengertia...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...","Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,"01 Mar, 2024 18 Nov, 2020 08 Okt, 2020 02 Agt,...",['[1] Pasal 1 angka 1 Undang-Undang Nomor 23 T...,https://www.hukumonline.com/klinik/a/kdrt-hing...
3,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,"22 Jul, 2024 12 Jun, 2024 19 Apr, 2024 09 Okt,...",['[1] Pasal 624 Undang-Undang Nomor 1 Tahun 20...,https://www.hukumonline.com/klinik/a/pacar-tid...
4,"Istri Siri Selingkuh, Bisakah Dipidana?",Saya seorang pria yang sudah beristri. Sayangn...,Perselingkuhan yang melibatkan pihak yang suda...,Artikel ini dibuat berdasarkan KUHP lama dan U...,['[1] Pasal 1 Undang-Undang Nomor 1 Tahun 1974...,https://www.hukumonline.com/klinik/a/istri-sir...
...,...,...,...,...,...,...
1113,Membayar Lomba Lari dengan Mata Uang Asing,Saya pernah dengar ada kewajiban menggunakan m...,NaN,Jika lomba lari tersebut diselenggarakan di In...,['Undang-Undang Nomor 7 Tahun 2011 tentang Mat...,https://www.hukumonline.com/klinik/a/membayar-...
1114,Larangan Jual Beli Darah,Apakah perdagangan organ tubuh dilarang? Tapi ...,NaN,"14 Agt, 2024 10 Mar, 2014 11 Apr, 2013 31 Okt,...",['Undang-Undang Nomor 36 Tahun 2009 tentang Ke...,https://www.hukumonline.com/klinik/a/larangan-...
1115,Menjerat Pelaku Kampanye Hitam di Media Sosial,Mulai memanasnya situasi menjelang PilPres 201...,NaN,Kampanye dalam Pemilihan Umum Presiden (Pilpre...,['Undang-Undang No. 11 Tahun 2008 tentang Info...,https://www.hukumonline.com/klinik/a/menjerat-...
1116,Eksekusi Hukuman Mati Bagi Terpidana yang Hamil,Jika ada kasus seorang isteri perwira TNI AD m...,NaN,"04 Jul, 2024 30 Apr, 2024 17 Jan, 2023 23 Des,...","['Kitab Undang-Undang Hukum Pidana', 'Undang-U...",https://www.hukumonline.com/klinik/a/eksekusi-...


In [ ]:
# import re
# def find_lb(text):
#     # Compile the regex pattern
#     pattern = re.compile(r'(Pasal\s+\d+(?:\s+ke-\d+|\s+ayat\s*\(\d+\))?|UU\s+No\.\s+\d+\s+Tahun\s+\d+)(?:\s+(\w+))?(?:\s+(\w+))?(?:\s+(\w+))?(?:\s+(\w+))?(?:\s+(\w+))?')

#     matches = pattern.findall(text)
#     filtered = set([tuple(filter(None, item)) for item in matches])
#     return filtered

#     # Test strings
# text = clean_df['long_answer'][2]
# find_lb(text)

## Extract individual lb

### Assign Id

In [ ]:
import pandas as pd
import ast
import re

def clean_lb(text):
    if not isinstance(text, str):
        raise ValueError("Input must be a string")
    # Combine regular expressions for efficiency
    filtered_spc = re.sub(r"\\xa0", ' ', text)
    clean = re.sub(r"\[(\d+)\]", '', filtered_spc)
    return clean.strip()

# Create a dictionary to store unique text values and their ids
text_to_id = {}
current_id = 1

# Function to convert list of strings to list of dictionaries with ids
def convert_to_dict_with_ids(text_list):
    global current_id, text_to_id
    result = []
    for text in text_list:
        if text not in text_to_id:
            text_to_id[text] = current_id
            current_id += 1
        result.append({'id': text_to_id[text], 'text': text})
    return result

In [ ]:
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.2_1118.csv', index_col=0)

df['legal_basis'] = df['legal_basis'].apply(clean_lb)
# convert string representations of lists to actual lists
df['legal_basis'] = df['legal_basis'].apply(ast.literal_eval)
# Apply the conversion function to the column
df['legal_basis'] = df['legal_basis'].apply(convert_to_dict_with_ids)
df

,topic,question,short_answer,long_answer,legal_basis,link
0,Jerat Pasal Penyalahgunaan Dana Donasi,Apa pasal yang bisa menjerat pengurus yayasan ...,Penyalahgunaan dana donasi oleh pengurus yayas...,Untuk menjawab pertanyaan Anda mengenai apakah...,"[{'id': 1, 'text': ' Penjelasan Pasal 32 ayat ...",https://www.hukumonline.com/klinik/a/jerat-pas...
1,Pengertian,Bagaimana ketentuan overmacht dalam hukum pida...,Daya paksa atau overmacht dalam hukum pidana m...,"11 Jan, 2023 15 Jul, 2022 10 Jul, 2020 22 Okt,...","[{'id': 9, 'text': ' R. Sugandhi. Kitab Undang...",https://www.hukumonline.com/klinik/a/pengertia...
2,"KDRT Hingga Meninggal, Penganiayaan atau Pembu...","Belakangan ini, viral kasus seorang suami di S...",Pasal KDRT atau kekerasan dalam rumah tangga y...,"01 Mar, 2024 18 Nov, 2020 08 Okt, 2020 02 Agt,...","[{'id': 17, 'text': ' Pasal 1 angka 1 Undang-U...",https://www.hukumonline.com/klinik/a/kdrt-hing...
3,"Pacar Tidak Mau Bertanggung Jawab, Kapan Maksi...",Saya ingin menanyakan bila laki-laki dan perem...,Jika terjadi persetubuhan antara sepasang keka...,"22 Jul, 2024 12 Jun, 2024 19 Apr, 2024 09 Okt,...","[{'id': 4, 'text': ' Pasal 624 Undang-Undang N...",https://www.hukumonline.com/klinik/a/pacar-tid...
4,"Istri Siri Selingkuh, Bisakah Dipidana?",Saya seorang pria yang sudah beristri. Sayangn...,Perselingkuhan yang melibatkan pihak yang suda...,Artikel ini dibuat berdasarkan KUHP lama dan U...,"[{'id': 26, 'text': ' Pasal 1 Undang-Undang No...",https://www.hukumonline.com/klinik/a/istri-sir...
...,...,...,...,...,...,...
1113,Membayar Lomba Lari dengan Mata Uang Asing,Saya pernah dengar ada kewajiban menggunakan m...,NaN,Jika lomba lari tersebut diselenggarakan di In...,"[{'id': 5463, 'text': 'Undang-Undang Nomor 7 T...",https://www.hukumonline.com/klinik/a/membayar-...
1114,Larangan Jual Beli Darah,Apakah perdagangan organ tubuh dilarang? Tapi ...,NaN,"14 Agt, 2024 10 Mar, 2014 11 Apr, 2013 31 Okt,...","[{'id': 5465, 'text': 'Undang-Undang Nomor 36 ...",https://www.hukumonline.com/klinik/a/larangan-...
1115,Menjerat Pelaku Kampanye Hitam di Media Sosial,Mulai memanasnya situasi menjelang PilPres 201...,NaN,Kampanye dalam Pemilihan Umum Presiden (Pilpre...,"[{'id': 5467, 'text': 'Undang-Undang No. 11 Ta...",https://www.hukumonline.com/klinik/a/menjerat-...
1116,Eksekusi Hukuman Mati Bagi Terpidana yang Hamil,Jika ada kasus seorang isteri perwira TNI AD m...,NaN,"04 Jul, 2024 30 Apr, 2024 17 Jan, 2023 23 Des,...","[{'id': 5469, 'text': 'Kitab Undang-Undang Huk...",https://www.hukumonline.com/klinik/a/eksekusi-...


In [ ]:
df.to_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.2_{len(df)}.csv')
print(f'Saved to clean_v0.2_{len(df)}.csv')

Saved to clean_v0.2_1118.csv


## Assign type of lb

In [ ]:
import pandas as pd
import ast

In [ ]:

# Function to explode the dictionaries into separate rows
def explode_dict_column(df, column_name):
    # Explode the list of dictionaries so each dictionary gets its own row
    exploded_df = df.explode(column_name)

    # Convert the dictionaries to separate columns
    dict_df = pd.json_normalize(exploded_df[column_name])

    # Combine with original dataframe's other columns if needed
    # Get all columns except the one we're exploding
    other_columns = [col for col in df.columns if col != column_name]

    if other_columns:
        # Reset index to maintain proper alignment
        exploded_df = exploded_df.reset_index(drop=True)
        result_df = pd.concat([exploded_df[other_columns], dict_df], axis=1)
    else:
        result_df = dict_df

    return result_df



In [ ]:
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/clean_v0.2_1118.csv', index_col=0)

df['legal_basis'] = df['legal_basis'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Create new dataframe
lb_df = pd.DataFrame(df['legal_basis'].explode().tolist())
lb_df

,id,text
0,1,Penjelasan Pasal 32 ayat (1) Undang-Undang No...
1,2,Pasal 1 angka 15 Undang-Undang Nomor 15 Tahun...
2,3,Pasal 1 angka 1 Undang-Undang 16 Tahun 2001 t...
3,4,Pasal 624 Undang-Undang Nomor 1 Tahun 2023 te...
4,5,Pasal 3 Peraturan Mahkamah Agung Nomor 2 Tahu...
...,...,...
7260,5470,Undang-Undang Nomor 35 Tahun 2009 tentang Nark...
7261,5471,Penetapan Presiden yang Kemudian Menjadi Undan...
7262,5472,Peraturan Kapolri No. 12 Tahun 2010 tentang Ta...
7263,5473,Peraturan Daerah Provinsi Daerah Khusus Ibukot...


In [ ]:
#drop duplicates
lb_df = lb_df.drop_duplicates()
lb_df = lb_df.reset_index(drop=True)
lb_df

,id,text
0,1,Penjelasan Pasal 32 ayat (1) Undang-Undang No...
1,2,Pasal 1 angka 15 Undang-Undang Nomor 15 Tahun...
2,3,Pasal 1 angka 1 Undang-Undang 16 Tahun 2001 t...
3,4,Pasal 624 Undang-Undang Nomor 1 Tahun 2023 te...
4,5,Pasal 3 Peraturan Mahkamah Agung Nomor 2 Tahu...
...,...,...
5469,5470,Undang-Undang Nomor 35 Tahun 2009 tentang Nark...
5470,5471,Penetapan Presiden yang Kemudian Menjadi Undan...
5471,5472,Peraturan Kapolri No. 12 Tahun 2010 tentang Ta...
5472,5473,Peraturan Daerah Provinsi Daerah Khusus Ibukot...


In [ ]:
lb_types = [['Peraturan Perundang-undangan Pusat','Undang-undang','UU'],
    ['Undang-undang Darurat','UUdrt'],
    ['Peraturan Pemerintah Pengganti Undang-Undang','Perpu'],
    ['Peraturan Pemerintah','PP'],
    ['Peraturan Presiden','Perpres'],
    ['Keputusan Presiden','Keppres'],
    ['Instruksi Presiden','Inpres'],
    ['Peraturan Perundang-undangan Daerah','Peraturan Daerah','PERDA '],
    ['Qanun'],
    ['Peraturan Daerah Khusus Provinsi Papua','PERDASUS PAPUA'],
    ['Peraturan Daerah Istimewa','Perdais'],
    ['Peraturan Gubernur','PERGUB'],
    ['Peraturan Bupati','PERBUP'],
    ['Peraturan Walikota','PERWALI'],
    ['Peraturan Badan Pemeriksa Keuangan', 'Permen BPK'],
    ['Peraturan Menteri Koordinator Bidang Kesejahteraan Rakyat','Permenko KESRA'],
    ['Keputusan Badan Pemeriksa Keuangan','Keputusan BPK'],
    ['Peraturan Menteri Dalam Negeri','PERMENDAGRI'],
    ['Peraturan Menteri Keuangan','PMK'],
    ['Peraturan Menteri Koordinator Bidang Politik, Hukum, dan Keamanan','PERMENKOPOLHUKAM','PERMENKO POLHUKAM'],
    ['Peraturan Menteri Luar Negeri','PERMENLU'],
    ['Peraturan Menteri Hukum dan HAM','PERMENKUMHAM'],
    ['Peraturan Menteri Pertahanan','PERMENHAN'],
    ['Peraturan Menteri Perhubungan','PERMENHUB'],
    ['Peraturan Kejaksaan Republik Indonesia','PERJA RI'],
    ['Peraturan Kepala Kepolisian Negara Republik Indonesia', 'Peraturan Kapolri','PERKAPOLRI','PERKA POLRI'],
    ['Peraturan Kepala Badan Narkotika Nasional','PERKA BNN'],
    ['Peraturan Kepala Badan Meteorologi, Klimatologi, dan Geofisika', 'PERKA BMKG'],
    ['Peraturan Badan Siber dan Sandi Negara','Peraturan BSSN'],
    ['Peraturan Komisi Nasional Hak Asasi Manusia','Peraturan KOMNAS HAM'],
    ['Peraturan Komisi Pemberantasan Korupsi','Peraturan KPK'],
    ['Peraturan Komisi Pemilihan Umum','Peraturan KPU'],
    ['Peraturan Badan Nasional Penanggulangan Terorisme','Peraturan BNPT'],
    ['Peraturan Badan Pengawas Pemilihan Umum','Peraturan BPPU'],
    ['Peraturan Badan Koordinasi Penanaman Modal','Peraturan BKPM'],
    ['Keputusan Menteri Keuangan','KMK'],
    ['Peraturan Menteri Perdagangan','Permendag'],
    ['Peraturan Menteri Perindustrian','Permenperin'],
    ['Peraturan Menteri Negara Perencanaan Pembangunan Nasional/Kepala Badan Perencanaan Pembangunan Nasional','Permen PPN/Kepala Bappenas'],
    ['Peraturan Menteri Koperasi dan Usaha Kecil dan Menengah','Permenkop UKM'],
    ['Peraturan Badan Pusat Statistik','Peraturan BPS'],
    ['Peraturan Kepala Badan Pusat Statistik','Peraturan Kepala BPS'],
    ['Peraturan Bank Indonesia','Peraturan BI'],
    ['Peraturan Otoritas Jasa Keuangan','Peraturan OJK'],
    ['Peraturan Kepala Pusat Pelaporan dan Analisis Transaksi Keuangan','Peraturan Kepala PPATK'],
    ['Peraturan Lembaga Penjamin Simpanan','Peraturan LPS'],
    ['Peraturan Badan Standardisasi Nasional','Peraturan BSN'],
    ['Keputusan Kepala Badan Standardisasi Nasional','Keputusan Kepala BSN',],
    ['Peraturan Kepala Lembaga Kebijakan Pengadaan Barang/Jasa Pemerintah','Perka LKPP'],
    ['Peraturan Komisi Pengawas Persaingan Usaha','Peraturan KPPU'],
    ['Keputusan Komisi Pengawas Persaingan Usaha','Keputusan KPPU'],
    ['Peraturan Badan Tenaga Nuklir Nasional','Peraturan BATAN'],
    ['Peraturan Dewan Perwakilan Rakyat','Peraturan DPR'],
    ['Peraturan Dewan Perwakilan Daerah','Peraturan DPD'],
    ['Peraturan Mahkamah Agung','Peraturan MA'],
    ['Peraturan Perpustakaan Nasional','Peraturan Perpusnas'],
    ['Peraturan Mahkamah Konstitusi','Peraturan MK'],
    ['Peraturan Komisi Yudisial','Peraturan KY'],
    ['Peraturan Menteri Koordinator Bidang Pembangunan Manusia Dan Kebudayaan','Permenko PMK'],
    ['Peraturan Menteri Sekretaris Negara Republik Indonesia','Permensesneg'],
    ['Peraturan Menteri Sosial','Permensos'],
    ['Peraturan Menteri Pariwisata Dan Ekonomi Kreatif','Permenparenkraf'],
    ['Peraturan Menteri Ketenagakerjaan','Permenaker'],
    ['Peraturan Menteri Komunikasi dan Informatika','Permenkominfo'],
    ['Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi','Permenpan RB','Permen PANRB'],
    ['Peraturan Menteri Pemberdayaan Perempuan Dan Perlindungan Anak','Permen PPPA'],
    ['Peraturan Menteri Pemuda dan Olahraga','Permenpora'],
    ['Peraturan Menteri Riset, Teknologi, Dan Pendidikan Tinggi Republik Indonesia','Permen Ristekdikti'],
    ['Peraturan Menteri Agraria dan Tata Ruang/ Kepala Badan Pertanahan Nasional Republik Indonesia','Permen Agraria/Kepala BPN','Permen ATR/KBPN'],
    ['Peraturan Menteri Desa, Pembangunan Daerah Tertinggal, dan Transmigrasi','Permendesa PDTT'],
    ['Peraturan Badan Pengawas Tenaga Nuklir','Peraturan BAPETEN'],
    ['Peraturan Kepala Badan Tenaga Nuklir Nasional'],
    ['Peraturan Lembaga Ilmu Pengetahuan Indonesia','Peraturan LIPI'],
    ['Peraturan Kepala Perpustakaan Nasional','Peraturan Kepala Perpusnas'],
    ['Peraturan Kepala Badan Nasional Penanggulangan Bencana','Peraturan Kepala BNPB'],
    ['Peraturan Badan Riset dan Inovasi Nasional','Peraturan BRIN'],
    ['Peraturan Kepala Badan Kependudukan dan Keluarga Berencana Nasional','Peraturan Kepala BKKBN'],
    ['Peraturan Kepala Badan Kepegawaian Negara','Peraturan Kepala BKN'],
    ['Peraturan Kepala Badan Pengawasan Keuangan dan Pembangunan','Peraturan Kepala BPKP'],
    ['Peraturan Kepala Lembaga Administrasi Negara','Peraturan LAN'],
    ['Peraturan Kepala Arsip Nasional Republik Indonesia','Peraturan ANRI'],
    ['Peraturan Badan Nasional Penempatan dan Perlindungan Tenaga Kerja Indonesia','Peraturan BNP2TKI'],
    ['Peraturan Direktur Utama Lembaga Penyiaran Publik Radio Republik Indonesia','Peraturan Dirut LPP RRI'],
    ['Peraturan Direksi Lembaga Penyiaran Publik Radio Republik Indonesia'],
    ['Peraturan Badan Informasi Geospasial','Peraturan BIG'],
    ['Peraturan Ombudsman Republik Indonesia','Peraturan Ombudsman RI'],
    ['Peraturan Ketua Ombudsman Republik Indonesia','Peraturan Ketua Ombudsman RI'],
    ['Peraturan Kepala Badan Ekonomi Kreatif','Peraturan Kepala Bekraf'],
    ['Peraturan Menteri Koordinator Bidang Kemaritiman dan Investasi','Permenko Maritim dan Investasi'],
    ['Peraturan Menteri Pertanian','Permentan'],
    ['Keputusan Menteri Pertanian','Keputusan Mentan'],
    ['Peraturan Menteri Kelautan dan Perikanan','Permen KKP'],
    ['Keputusan Menteri Kelautan dan Perikanan','Kepmen KKP'],
    ['Peraturan Menteri Energi dan Sumber Daya Mineral','Permen ESDM'],
    ['Keputusan Menteri Energi dan Sumber Daya Mineral','Kepmen ESDM'],
    ['Peraturan Menteri Pekerjaan Umum dan Perumahan Rakyat','Permen PUPR'],
    ['Peraturan Menteri Lingkungan Hidup','Permen LH'],
    ['Peraturan Menteri Lingkungan Hidup Dan Kehutanan','Permen LHK'],
    ['Keputusan Kepala Satuan Kerja Khusus Pelaksana Kegiatan Usaha Hulu Minyak dan Gas Bumi','Keputusan Kepala SKK Migas'],
    ['Peraturan Badan Kependudukan dan Keluarga Berencana Nasional','Peraturan BKKBN'],
    ['Instruksi Menteri Dalam Negeri','Instruksi Mendagri'],
    ['Keputusan Menteri Dalam Negeri','Kepmendagri'],
    ['Peraturan Menteri Agama','Peraturan Menag'],
    ['Peraturan Kepala Badan Pengusahaan Kawasan Perdagangan dan Pelabuhan Bebas Sabang'],
    ['Peraturan Kepala Badan Pengusahaan Kawasan Perdagangan Bebas dan Pelabuhan Bebas Batam'],
    ['Peraturan Badan Nasional Pengelola Perbatasan','Peraturan BNPP'],
    ['Peraturan Menteri Kesehatan','Permenkes'],
    ['Keputusan Menteri Kesehatan','Kepmenkes'],
    ['Surat Edaran Menteri Kesehatan','Surat Edaran Menkes'],
    ['Peraturan Menteri Pendidikan dan Kebudayaan','Permendikbud'],
    ['Peraturan Badan Penyelenggara Jaminan Sosial Kesehatan','Peraturan BPJS Kesehatan'],
    ['Peraturan BPJS Ketenagakerjaan'],
    ['Peraturan Kepala Badan Pengawas Obat dan Makanan','Perka BPOM'],
    ['Peraturan Menteri Badan Usaha Milik Negara','Permen BUMN'],
    ['Surat Edaran Menteri Badan Usaha Milik Negara','SE Menteri BUMN'],
    ['Peraturan Menteri Koordinator Bidang Perekonomian','Permenko Perekonomian'],
    ['Peraturan Badan Kepegawaian Negara','Peraturan BKN'],
    ['Peraturan Lembaga Kebijakan Pengadaan Barang/Jasa Pemerintah','Peraturan LKPP'],
    ['Surat Edaran Otoritas Jasa Keuangan','SE OJK','Surat Edaran OJK'],
    ['Surat Edaran Kepala Lembaga Kebijakan Pengadaan Barang/Jasa Pemerintah','Surat Edaran Kepala LKPP','SE Kepala SKPP'],
    ['Peraturan Lembaga Perlindungan Saksi dan Korban','Peraturan LPSK'],
    ['Peraturan Jaksa Agung'],
    ['Peraturan Arsip Nasional Republik Indonesia','Peraturan ANRI'],
    ['Peraturan Ketua Komisi Nasional Hak Asasi Manusia','Peraturan Komnas HAM'],
    ['Peraturan Badan Pengawas Obat dan Makanan','Peraturan BPOM'],
    ['Peraturan Badan Meteorologi, Klimatologi, dan Geofisika','Peraturan BMKG'],
    ['Peraturan Lembaga Administrasi Negara','Peraturan LAN'],
    ['Peraturan Gubernur Lembaga Ketahanan Nasional','Pergub Lemhannas'],
    ['Peraturan Kepala Badan Keamanan Laut','Perka Bakamla'],
    ['Peraturan Badan Keamanan Laut','Peraturan Bakamla'],
    ['Surat Edaran Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi','SE Menteri PAN-RB','SE Menpan RB'],
    ['Peraturan Sekretaris Kabinet'],
    ['Peraturan Badan Nasional Penanggulangan Bencana','Peraturan BNPB'],
    ['Keputusan Kepala Badan Nasional Penanggulangan Bencana'],
    ['Peraturan Kepolisian Negara Republik Indonesia'],
    ['Peraturan Menteri Pendidikan, Kebudayaan, Riset, dan Teknologi'],
    ['Peraturan Badan Pengawasan Keuangan dan Pembangunan'],
    ['Peraturan Badan Intelijen Negara','Peraturan BIN'],
    ['Peraturan Pusat Pelaporan dan Analisis Transaksi Keuangan','Peraturan PPATK'],
    ['Peraturan Majelis Permusyawaratan Rakyat','Peraturan MPR'],
    ['Surat Edaran Menteri Dalam Negeri','SE Mendagri'],
    ['Keputusan Menteri Luar Negeri','Kepmenlu'],
    ['Peraturan Badan Narkotika Nasional','Peraturan BNN'],
    ['Peraturan Kepala Badan Koordinasi Penanaman Modal','Peraturan BPKM'],
    ['Peraturan Kepala Badan Standardisasi Nasional','Perka BSN'],
    ['Peraturan Kepala Badan Pengawas Tenaga Nuklir','Perka BAPETEN'],
    ['Peraturan Kepala Badan Informasi Geospasial','Perka BIG'],
    ['Peraturan Badan Pembinaan Ideologi Pancasila','Peraturan BPIP'],
    ['Peraturan Dewan Direksi Lembaga Penyiaran Publik Televisi Republik Indonesia','Peraturan Direksi LPP TVRI'],
    ['Keputusan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi','Kepmen PAN-RB','Kepmen PANRB'],
    ['Keputusan Menteri Sosial','Kepmensos'],
    ['Peraturan Badan Nasional Pencarian dan Pertolongan','Peraturan BNPP'],
    ['Peraturan Badan Pelindungan Pekerja Migran Indonesia','Peraturan BP2MI'],
    ['Peraturan Sekretariat Jenderal Dewan Ketahanan Nasional','Peraturan Setjen Wantanas'],
    ['Peraturan Kepala Badan Nasional Penanggulangan Terorisme','Perka BNPT'],
    ['Peraturan Badan Pengelola Tabungan Perumahan Rakyat','Peraturan BP Tapera'],
    ['Peraturan Badan Pengelola Keuangan Haji','Peraturan BPKH'],
    ['Peraturan Badan Ekonomi Kreatif','Peraturan BEKRAF'],
    ['Kitab Undang-Undang Hukum Pidana','KUHP'],
    ['Kitab Undang-Undang Hukum Perdata','KUH Perdata'],
    ['Kitab Undang-Undang Hukum Acara Pidana','KUHAP'],
    ['Putusan Mahkamah Agung', 'Putusan MA'],
    ['Putusan Mahkamah Konstitusi', 'Putusan MK']
]

In [ ]:
def classify_lb(lb_value):
    # Check if any keywords from lb_types are present in lb_value
    for type_list in lb_types:
        sorted_type_list = sorted(type_list, key=len, reverse=True)
        for lb_type in sorted_type_list:
            if lb_type.lower() in lb_value.lower():
                return type_list[0]  # Return the first matching keyword

    # If no match is found, return "Unclassified"
    return "Unclassified"

In [ ]:
lb_df['type'] = lb_df['text'].apply(classify_lb)
lb_df


,id,text,type
0,1,Penjelasan Pasal 32 ayat (1) Undang-Undang No...,Peraturan Perundang-undangan Pusat
1,2,Pasal 1 angka 15 Undang-Undang Nomor 15 Tahun...,Peraturan Perundang-undangan Pusat
2,3,Pasal 1 angka 1 Undang-Undang 16 Tahun 2001 t...,Peraturan Perundang-undangan Pusat
3,4,Pasal 624 Undang-Undang Nomor 1 Tahun 2023 te...,Peraturan Perundang-undangan Pusat
4,5,Pasal 3 Peraturan Mahkamah Agung Nomor 2 Tahu...,Peraturan Mahkamah Agung
...,...,...,...
5469,5470,Undang-Undang Nomor 35 Tahun 2009 tentang Nark...,Peraturan Perundang-undangan Pusat
5470,5471,Penetapan Presiden yang Kemudian Menjadi Undan...,Peraturan Perundang-undangan Pusat
5471,5472,Peraturan Kapolri No. 12 Tahun 2010 tentang Ta...,Peraturan Kepala Kepolisian Negara Republik In...
5472,5473,Peraturan Daerah Provinsi Daerah Khusus Ibukot...,Peraturan Perundang-undangan Daerah


In [ ]:
import pandas as pd
lb_df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.1_5474.csv', index_col=0)
lb_df['type'] = lb_df['text'].apply(classify_lb)
lb_df.to_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.1_{len(lb_df)}.csv')
print(f'Saved to lb_v0.1_{len(lb_df)}.csv')

Saved to lb_v0.1_5474.csv


In [ ]:
print(lb_df['type'].value_counts())

type
Peraturan Perundang-undangan Pusat                                         2592
Unclassified                                                               1061
Peraturan Pemerintah                                                        412
Kitab Undang-Undang Hukum Acara Pidana                                      361
Peraturan Kepala Kepolisian Negara Republik Indonesia                       213
Peraturan Menteri Hukum dan HAM                                             187
Kitab Undang-Undang Hukum Pidana                                            126
Peraturan Perundang-undangan Daerah                                         108
Peraturan Mahkamah Agung                                                     69
Peraturan Gubernur                                                           52
Peraturan Menteri Kesehatan                                                  50
Peraturan Menteri Perhubungan                                                43
Peraturan Menteri Pendidikan dan Ke

In [ ]:
# Merge the two DataFrames
def replace_rows(df1, df2):
    # Verify both dataframes have the same columns
    if not all(df1.columns == df2.columns):
        raise ValueError("Both dataframes must have the same columns")

    # Create a copy of df1 to avoid modifying the original
    result_df = df1.copy()

    # Replace rows where indexes match
    result_df.update(df2)

    # Return the number of rows updated and the updated dataframe
    num_updates = len(df2.index.intersection(df1.index))
    return num_updates, result_df

In [ ]:
# manual fix
import pandas as pd
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.1_5474.csv', index_col=0)
fixed_df = pd.read_csv('/content/unclass - unclass.csv.csv', index_col=0)
num_updated, updated_lb_df = replace_rows(df, fixed_df)
print(f"Number of rows updated: {num_updated}")
print("\nUpdated DataFrame:")
updated_lb_df

Number of rows updated: 1061

Updated DataFrame:


,text,type,pasal,angka,huruf,nomor,tahun
id,,,,,,,
1,Penjelasan Pasal 32 ayat (1) Undang-Undang No...,Peraturan Perundang-undangan Pusat,32.0,1.0,NaN,31.0,1999.0
2,Pasal 1 angka 15 Undang-Undang Nomor 15 Tahun...,Peraturan Perundang-undangan Pusat,1.0,15.0,NaN,15.0,2006.0
3,Pasal 1 angka 1 Undang-Undang 16 Tahun 2001 te...,Peraturan Perundang-undangan Pusat,1.0,1.0,NaN,16.0,2001.0
4,Pasal 624 Undang-Undang Nomor 1 Tahun 2023 te...,Peraturan Perundang-undangan Pusat,624.0,NaN,NaN,1.0,2023.0
5,Pasal 3 Peraturan Mahkamah Agung Nomor 2 Tahu...,Peraturan Mahkamah Agung,3.0,NaN,NaN,2.0,2012.0
...,...,...,...,...,...,...,...
5470,Undang-Undang Nomor 35 Tahun 2009 tentang Nark...,Peraturan Perundang-undangan Pusat,NaN,NaN,NaN,35.0,2009.0
5471,Penetapan Presiden yang Kemudian Menjadi Undan...,Peraturan Perundang-undangan Pusat,NaN,NaN,NaN,2.0,1964.0
5472,Peraturan Kapolri No. 12 Tahun 2010 tentang Ta...,Peraturan Kepala Kepolisian Negara Republik In...,NaN,NaN,NaN,12.0,2010.0


In [ ]:
updated_lb_df.to_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.2_{len(updated_lb_df)}.csv')

In [ ]:
import re

def parse_legal_text(text):
    # First try to match combined nomor/tahun format (X/Y)
    combined_pattern = r'(\d+)/(\d{4})'
    combined_match = re.search(combined_pattern, text)
    # Initialize nomor and tahun from combined format if found
    nomor = combined_match.group(1) if combined_match else None
    tahun = combined_match.group(2) if combined_match else None

    # Individual patterns for each group
    patterns = {
        'pasal': r'(?:Pasal|pasal) (\d+)',
        'angka': r'(?:angka|ayat) (?:\s*\()?(\d+)(?:\))?',
        'nomor': r'(?:Nomor|nomor|No.|no.) (\d+)',
        'tahun': r'Tahun (\d+)'
    }

    result = {
        'pasal': [],  # List to store all occurrences
        'angka': [],   # List to store all occurrences
        'nomor': nomor,  # Pre-filled from combined format if found
        'tahun': tahun,  # Pre-filled from combined format if found
    }

    # Search for each pattern independently using re.findall
    for key, pattern in patterns.items():
        # Skip nomor and tahun if already found in combined format
        if (key in ['nomor', 'tahun']) and result[key] is not None:
            continue
        matches = re.findall(pattern, text)
        if matches:
            result[key] = matches  # Store all occurrences in a list
    # cast to int if list has only 1 items
    for key, value in result.items():
        if isinstance(value, list) and len(value) == 1:
            result[key] = int(value[0])
    return result


In [ ]:
import pandas as pd
parsed_lb_df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.8_5474.csv', index_col=0)
parsed_lb_df.drop(columns=['huruf'], inplace=True)
# First, set all columns to object dtype
parsed_lb_df = parsed_lb_df.astype(object)

for index, row in parsed_lb_df.iterrows():
    parsed_text = parse_legal_text(row['text'])
    for key, value in parsed_text.items():
        # Check if the value is a list and if it is, join it into a string
        if isinstance(value, list):
            # If the list is empty, assign None
            parsed_lb_df.at[index, key] = ', '.join(map(str, value)) if value else None
        else:
            parsed_lb_df.at[index, key] = value
parsed_lb_df

,text,type,pasal,angka,nomor,tahun,fulltext
id,,,,,,,
1,Penjelasan Pasal 32 ayat (1) Undang-Undang Nom...,Peraturan Perundang-undangan Pusat,32,1,31,1999,['(1) Dalam hal penyidik menemukan dan berpend...
2,Pasal 1 angka 15 Undang-Undang Nomor 15 Tahun ...,Peraturan Perundang-undangan Pusat,1,15,15,2006,Keuangan Negara adalah keseluruhan kegiatan...
3,Pasal 1 angka 1 Undang-Undang 16 Tahun 2001 te...,Peraturan Perundang-undangan Pusat,1,1,None,2001,asal 1 Dalam Undang-undang ini yang dimaksud d...
4,Pasal 624 Undang-Undang Nomor 1 Tahun 2023 ten...,Peraturan Perundang-undangan Pusat,624,None,1,2023,NaN
5,Pasal 3 Peraturan Mahkamah Agung Nomor 2 Tahu...,Peraturan Mahkamah Agung,3,None,2,2012,NaN
...,...,...,...,...,...,...,...
5470,Undang-Undang Nomor 35 Tahun 2009 tentang Nark...,Peraturan Perundang-undangan Pusat,None,None,35,2009,NaN
5471,Penetapan Presiden yang Kemudian Menjadi Undan...,Penetapan Presiden,None,None,2,1964,Dengan tidak mengurangi ketentuan-ketentuan hu...
5472,Peraturan Kapolri No. 12 Tahun 2010 tentang Ta...,Peraturan Kepala Kepolisian Negara Republik In...,None,None,12,2010,NaN


In [ ]:
parsed_lb_df.to_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.9_{len(parsed_lb_df)}.csv')

In [ ]:
parsed_lb_df.to_csv(f'/content/drive/MyDrive/SKRIPSI/unclean_lb_{len(parsed_lb_df)}.csv')
print(f'Saved to unclean_lb_{len(parsed_lb_df)}.csv')

In [ ]:
#get data with missing infos to fix manually
import pandas as pd
df = pd.read_csv(f'/content/drive/MyDrive/SKRIPSI/lb_v0.2_5474.csv', index_col=0)
classified = df[(df['type'] != 'Unclassified') & (~df['type'].isin(['Kitab Undang-Undang Hukum Pidana', 'Kitab Undang-Undang Hukum Perdata', 'Kitab Undang-Undang Hukum Acara Pidana']))]
empty_no_or_thn = classified[((classified['tahun'].isna()) | (classified['nomor'].isna()))]
empty_no_or_thn.to_csv(f'empty_no_or_thn_{len(empty_no_or_thn)}.csv')
print(f'Saved to empty_no_or_thn_{len(empty_no_or_thn)}.csv')

Saved to empty_no_or_thn_1313.csv
